# Import

In [1]:
import sys
%matplotlib inline
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import regression_model
import NNModel
from tagging import *
from NNModel import *
from window_aggregation import *
from dull_model import *
from collocation import *

Using TensorFlow backend.


# Load test data

In [5]:
path_to_data = '../data/test.csv'
test_df = pd.read_csv(path_to_data)

# Init models

In [6]:
nn_model = nn_init('../model/')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
tag_model = tag_init('../model/dn/')

In [8]:
dull_model = DullModel('../model/rm/')

# Predict

In [11]:
products = test_df['PRODUCT'].unique()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
for product in products:
    comments = test_df[test_df['PRODUCT'] == product]['TEXT'].tolist()
    prod_info = test_df[test_df['PRODUCT'] == product].head(1)
    prod_name = prod_info['NAME'].tolist()[0].strip() if len(prod_info['NAME']) > 0 else ''
    print('Продукт %d, %s' % (product, prod_name))
    
    # DN - tags
    num_chosen_tags = 3
    single_prod_df = test_df[test_df['PRODUCT'] == product]
    tags = tag_model.predict(single_prod_df, top_n=num_chosen_tags+1)
    tags = [t for t in tags if len(t.split(' ')) <=3]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    
    # coallocation tags
    num_chosen_coallocations = 3
    col_model = Collocation_find('../model/mk/stopw-1.csv', path_to_data)
    tags = col_model.predict(product)[:num_chosen_coallocations]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    print(' ')
    
    # nn 
    wnd_agg = aggregate(comments, num_comments=3, model=nn_model, wnd_l=12, stride=6, positive_q=(0.8, 0.95), negative_threshold=3)
    print('\tПокупатели отмечают такие положительные моменты: ')
    if len(wnd_agg['+']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['+']:
            print('\t\t...%s...' % com)
    print(' ')
            
    print('\tПокупатели отмечают такие отрицательные моменты: ')
    if len(wnd_agg['-']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['-']:
            print('\t\t...%s...' % com)
    print(' ')
    
    # comments
    num_chosen_comments = 3
    dull_pres = dull_model.predict(single_prod_df, list(range(len(single_prod_df))))[:num_chosen_comments]
    print('\tПокупатели также находят важным: ')
    if len(dull_pres) == 0:
        print('\t\t---')
    else:
        for com in dull_pres:
            print('\t\t...%s...' % com)
    
    print(' ')        
    print(''.join(['*'] * 110))
    print(' ')

Продукт 10000838, Автомобильная магнитола с CD MP3 Pioneer DEH-9300SD
	#аппарат_отличный 
	#незадача_шипение #белая_подсветка #экран_читабелен 
 
	Покупатели отмечают такие положительные моменты: 
		...выглядит очень стильно и добротно ...
		...читает, что не может не радовать) главное прикупить пульт и про якобы ...
		...аппарат отличный ...
 
	Покупатели отмечают такие отрицательные моменты: 
		---
 
	Покупатели также находят важным: 
		...на  пример  поменять  акустические  провода  на  более  приличные  или  проверить  качество  подключение  может...
		...представитель  через  него  подключены  усилитель  на  четыре  колонки  и  сабвуфер  аппарат  отличный  а...
		...этот  аппарат  на  достойном  уровне...
 
**************************************************************************************************************
 
Продукт 10001253, Телевизор LG 32 LW4500
	#тв_цифра_ #телевизор_очень_хорош #изображение_отличное 
	#комп_видеомагнитофон #читаться_вроде #подключилось_проблем 
 
	П

	#брали_витрины_выветрился #выкручиваю_половинку_мощность #греемся_радуемся_запаха 
 
	Покупатели отмечают такие положительные моменты: 
		...75 выкручиваю на половинку мощность - и жара ...
 
	Покупатели отмечают такие отрицательные моменты: 
		---
 
	Покупатели также находят важным: 
		...сегодня  греемся  и  радуемся  запаха  почти  нет  брали  с  витрины  может  он  и...
		...половинку  мощность  и  жара  запах  сначала  стоял  сильный  сейчас  ок  советую...
 
**************************************************************************************************************
 
Продукт 20032486, Тепловентилятор Zanussi ZFH/S-201
	#тепловентилятор_очень_хороший 
	#хороший_шумный_компактный #марка_средней #аналоги_поверила_взяла 
 
	Покупатели отмечают такие положительные моменты: 
		...тепловентилятор очень хороший, есть три режима - холодный воздух (на лето), теплый ...
		...я лично даже не обращаю на него внимания ...
		...дома тоже пару раз включали, но тут обязательно надо позаботитьс

	
	#_ #_ #лучший_товар_года 
 
	Покупатели отмечают такие положительные моменты: 
		...лучший товар 2016 года для меня – это планшет планшет samsung galaxy ...
		...3) lenovo показывает наиболее честно, яркость близка к самсунгу - лучший экран ...
		...отличный планшет от samsung, в котором очень хороший экран ...
 
	Покупатели отмечают такие отрицательные моменты: 
		---
 
	Покупатели также находят важным: 
		...самсунга  стекло  идеальное  скользкое  легко  чистить  также  только  у  самсунга  есть  авторегулировка  яркости...
		...большой  планшет  я  занимаюсь  фотографией  потому  имею  большие  претензии  к  качеству  и  честности...
		...но  хорошо  что  отправился  посмотреть  в  магазине  увы  хваленый  амолед  дисплей  оказался  совершенно...
 
**************************************************************************************************************
 
Продукт 40056960, Игра для PS3 Медиа Killzone 3 Collectors Edition
	#игра_хорошая #звук_вообще_отличный #если_система_стои

IndexError: list index out of range